# 📊 Dataset Maker by Hollowstrawberry

This is based on the work of [Kohya-ss](https://github.com/kohya-ss/sd-scripts) and [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb). Thank you!

### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning inference.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Homepage** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |
| 🌟 **XL Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) |  |

In [12]:
%pip install ipywidgets

import os
import pathlib

import ipywidgets as widgets
from IPython import get_ipython
from IPython.display import Markdown, display

COLAB = False

if COLAB:
    from google.colab.output import clear as clear_output
else:
    from IPython.display import clear_output

display(
    widgets.HTML("<h2>🚩 Start Here</h2>"),
)

project_name_widget = widgets.Text(description="Project name:")
folder_structure_widget = widgets.Dropdown(
    options=[
        "Organize by category (MyDrive/lora_training/datasets/project_name)",
        "Organize by project (MyDrive/Loras/project_name/dataset)",
    ],
    value="Organize by project (MyDrive/Loras/project_name/dataset)",
    description="Folder structure:",
    disabled=False,
)


def proceed_step_1(b):
    global project_name
    project_name = project_name_widget.value
    project_name = project_name.strip()

    global folder_structure
    folder_structure = folder_structure_widget.value

    global project_base
    global project_subfolder
    global root_dir
    global deps_dir
    global main_dir
    global config_folder
    global images_folder
    global step1_installed_flag

    if (
        not project_name
        or any(c in project_name for c in " .()\"'\\")
        or project_name.count("/") > 1
    ):
        print("Please write a valid project_name.")
    else:
        if COLAB and not os.path.exists("/content/drive"):
            from google.colab import drive

            print("📂 Connecting to Google Drive...")
            drive.mount("/content/drive")

        project_base = (
            project_name
            if "/" not in project_name
            else project_name[: project_name.rfind("/")]
        )
        project_subfolder = (
            project_name
            if "/" not in project_name
            else project_name[project_name.rfind("/") + 1 :]
        )

        root_dir = "/content" if COLAB else pathlib.Path.home() / "Loras"
        deps_dir = os.path.join(root_dir, "deps")

        if "/Loras" in folder_structure:
            main_dir = (
                os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
            )
            config_folder = os.path.join(main_dir, project_base)
            images_folder = os.path.join(main_dir, project_base, "dataset")
            if "/" in project_name:
                images_folder = os.path.join(images_folder, project_subfolder)
        else:
            main_dir = (
                os.path.join(root_dir, "drive/MyDrive/lora_training")
                if COLAB
                else root_dir
            )
            config_folder = os.path.join(main_dir, "config", project_name)
            images_folder = os.path.join(main_dir, "datasets", project_name)

        for dir in [main_dir, deps_dir, images_folder, config_folder]:
            os.makedirs(dir, exist_ok=True)

        print(f"✅ Project {project_name} is ready!")
        step1_installed_flag = True


proceed1_btn = widgets.Button(
    description="Proceed",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Proceed",
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)
proceed1_btn.on_click(proceed_step_1)

display(
    widgets.HTML("<h2>1️⃣ Setup</h2>"),
    widgets.Label(
        "This cell will load some requirements and create the necessary folders in your Google Drive."
    ),
    widgets.Label(
        "Your project name can't contain spaces but it can contain a single / to make a subfolder in your dataset."
    ),
    project_name_widget,
    widgets.Label(
        "The folder structure doesn't matter and is purely for comfort. Make sure to always pick the same one. I like organizing by project."
    ),
    folder_structure_widget,
    proceed1_btn,
)

DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/setuptools-70.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


HTML(value='<h2>🚩 Start Here</h2>')

HTML(value='<h2>1️⃣ Setup</h2>')

Label(value='This cell will load some requirements and create the necessary folders in your Google Drive.')

Label(value="Your project name can't contain spaces but it can contain a single / to make a subfolder in your …

Text(value='', description='Project name:')

Label(value="The folder structure doesn't matter and is purely for comfort. Make sure to always pick the same …

Dropdown(description='Folder structure:', index=1, options=('Organize by category (MyDrive/lora_training/datas…

Button(description='Proceed', icon='check', style=ButtonStyle(), tooltip='Proceed')

In [13]:
import json
import time
from urllib.request import Request, urlopen

display(
    widgets.HTML(
        """
    <h2>2️⃣ Scrape images from Gelbooru</h2>
    <p>We will grab images from the popular anime gallery <a href='https://gelbooru.com/'>Gelbooru</a>. Images are sorted by tags, including poses, scenes, character traits, character names, artists, etc.</p>
    <ul>
        <li>If you instead want to use your own images, upload them to your Google Drive's `Loras/project_name/dataset` folder.</li>
        <li>If you instead want to download screencaps of anime episodes, try <a href='https://colab.research.google.com/drive/1oBSntB40BKzNmKceXUlkXzujzdQw-Ci7'>this other colab by another person</a>. It's more complicated though.</li>
    </ul>
    <p>Up to 1000 images may be downloaded by this step in just one minute. Don't abuse it.</p>
    <p>Your target tags should include the relevant tags for your character/concept/artstyle, and exclude undesired tags (for example, explicit images may affect learning).</p>
"""
    )
)

tags_widget = widgets.Text(description="tags")
max_resolution_widget = widgets.BoundedIntText(
    value=3072,
    min=1024,
    max=8196,
    step=1024,
    description="Max resolution:",
    disabled=False,
)

include_posts_with_parent_widget = widgets.Checkbox(
    value=True, description="Include posts with parent:", disabled=False, indent=False
)


def proceed_step_2(b):
    if "step1_installed_flag" not in globals():
        raise Exception("Please run step 1 first!")

    global tags
    tags = tags_widget.value
    global max_resolution
    max_resolution = 3072

    global include_posts_with_parent
    include_posts_with_parent = include_posts_with_parent_widget.value

    global step2_installed_flag

    tags = (
        tags.replace(" ", "+")
        .replace("(", "%28")
        .replace(")", "%29")
        .replace(":", "%3a")
        .replace("&", "%26")
    )
    url = "https://gelbooru.com/index.php?page=dapi&json=1&s=post&q=index&limit=100&tags={}".format(
        tags
    )
    user_agent = "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/93.0.4577.83 Safari/537.36"
    limit = 100  # hardcoded by gelbooru
    total_limit = 1000  # you can edit this if you want but I wouldn't recommend it
    supported_types = (".png", ".jpg", ".jpeg")

    def ubuntu_deps():
        print("🏭 Installing dependencies...\n")
        !apt -y install aria2
        return not get_ipython().__dict__["user_ns"]["_exit_code"]

    if "step2_installed_flag" not in globals():
        if ubuntu_deps():
            clear_output()
            step2_installed_flag = True
        else:
            print("❌ Error installing dependencies, attempting to continue anyway...")

    def get_json(url):
        with urlopen(Request(url, headers={"User-Agent": user_agent})) as page:
            return json.load(page)

    def filter_images(data):
        return [
            (
                p["file_url"]
                if p["width"] * p["height"] <= max_resolution**2
                else p["sample_url"]
            )
            for p in data["post"]
            if (p["parent_id"] == 0 or include_posts_with_parent)
            and p["file_url"].lower().endswith(supported_types)
        ]

    global count

    def download_images():
        data = get_json(url)
        count = data["@attributes"]["count"]

        if count == 0:
            print("📷 No results found")
            return

        print(f"🎯 Found {count} results")
        test_url = "https://gelbooru.com/index.php?page=post&s=list&tags={}".format(
            tags
        )
        display(Markdown(f"[Click here to open in browser!]({test_url})"))

        perform_download_btn = widgets.Button(
            description="Yes",
            disabled=False,
            button_style="",  # 'success', 'info', 'warning', 'danger' or ''
            tooltip="Yes",
            icon="check",  # (FontAwesome names without the `fa-` prefix)
        )

        cancel_dowload_btn = widgets.Button(
            description="No",
            disabled=False,
            button_style="",  # 'success', 'info', 'warning', 'danger' or ''
            tooltip="No",
            icon="check",  # (FontAwesome names without the `fa-` prefix)
        )

        def perform_download(b):
            print("📩 Grabbing image list...")

            data = get_json(url)
            count = data["@attributes"]["count"]

            image_urls = set()
            image_urls = image_urls.union(filter_images(data))
            for i in range(total_limit // limit):
                count -= limit
                if count <= 0:
                    break
                time.sleep(0.1)
                image_urls = image_urls.union(
                    filter_images(get_json(url + f"&pid={i+1}"))
                )

            scrape_file = os.path.join(config_folder, f"scrape_{project_subfolder}.txt")
            with open(scrape_file, "w") as f:
                f.write("\n".join(image_urls))

            print(f"🌐 Saved links to {scrape_file}\n\n🔁 Downloading images...\n")
            old_img_count = len(
                [
                    f
                    for f in os.listdir(images_folder)
                    if f.lower().endswith(supported_types)
                ]
            )

            os.chdir(images_folder)
            !aria2c --console-log-level=warn -c -x 16 -k "1M" -s 16 -i {scrape_file}

            new_img_count = len(
                [
                    f
                    for f in os.listdir(images_folder)
                    if f.lower().endswith(supported_types)
                ]
            )
            print(f"\n✅ Downloaded {new_img_count - old_img_count} images.")

        perform_download_btn.on_click(perform_download)
        cancel_dowload_btn.on_click(
            lambda: display(widgets.Markdown("❌ Download cancelled"))
        )

        display(
            widgets.Label(
                f"🔽 Will download to {images_folder.replace('/content/drive/', '')} (A confirmation box should appear below, otherwise run this cell again)"
            ),
            widgets.Label(
                "❓ Enter the word 'yes' if you want to proceed with the download: "
            ),
            perform_download_btn,
            cancel_dowload_btn,
        )

    download_images()


proceed2_btn = widgets.Button(
    description="Proceed",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Proceed",
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)
proceed2_btn.on_click(proceed_step_2)

display(
    widgets.Label(
        """Separate words with underscores, separate tags with spaces, and use - to exclude a tag. You can also include a minimum score: `score:>10`"""
    ),
    tags_widget,
    widgets.Label(
        "If an image is bigger than this resolution a smaller version will be downloaded instead."
    ),
    max_resolution_widget,
    proceed2_btn,
)

HTML(value="\n    <h2>2️⃣ Scrape images from Gelbooru</h2>\n    <p>We will grab images from the popular anime …

Label(value='Separate words with underscores, separate tags with spaces, and use - to exclude a tag. You can a…

Text(value='', description='tags')

Label(value='If an image is bigger than this resolution a smaller version will be downloaded instead.')

BoundedIntText(value=3072, description='Max resolution:', max=8196, min=1024, step=1024)

Button(description='Proceed', icon='check', style=ButtonStyle(), tooltip='Proceed')

In [14]:
similarity_threshold_widget = widgets.FloatText(
    value=3e-4, description="Similarity threshold:", disabled=False
)


def proceed_step_3(b):
    return

    if "step1_installed_flag" not in globals():
        raise Exception("Please run step 1 first!")

    return

    similarity_threshold = similarity_threshold_widget.value
    os.chdir(root_dir)
    model_name = "clip-vit-base32-torch"
    supported_types = (".png", ".jpg", ".jpeg")
    img_count = len(os.listdir(images_folder))
    batch_size = min(250, img_count)

    if "step3_installed_flag" not in globals():
        print("🏭 Installing dependencies...\n")
        %pip -q install fiftyone ftfy
        %pip -q install fiftyone-db-ubuntu2204
        if not get_ipython().__dict__["user_ns"]["_exit_code"]:
            clear_output()
            step3_installed_flag = True
        else:
            print("❌ Error installing dependencies, attempting to continue anyway...")

    import fiftyone as fo
    import fiftyone.zoo as foz
    import numpy as np
    from fiftyone import ViewField as F
    from sklearn.metrics.pairwise import cosine_similarity

    non_images = [
        f for f in os.listdir(images_folder) if not f.lower().endswith(supported_types)
    ]
    if non_images:
        print(
            f"💥 Error: Found non-image file {non_images[0]} - This program doesn't allow it. Sorry! Use the Extras at the bottom to clean the folder."
        )
    elif img_count == 0:
        print(f"💥 Error: No images found in {images_folder}")
    else:
        print("\n💿 Analyzing dataset...\n")
        dataset = fo.Dataset.from_dir(
            images_folder, dataset_type=fo.types.ImageDirectory
        )
        model = foz.load_zoo_model(model_name)
        embeddings = dataset.compute_embeddings(model, batch_size=batch_size)

        batch_embeddings = np.array_split(embeddings, batch_size)
        similarity_matrices = []
        max_size_x = max(array.shape[0] for array in batch_embeddings)
        max_size_y = max(array.shape[1] for array in batch_embeddings)

        for i, batch_embedding in enumerate(batch_embeddings):
            similarity = cosine_similarity(batch_embedding)
            # Pad 0 for np.concatenate
            padded_array = np.zeros((max_size_x, max_size_y))
            padded_array[0 : similarity.shape[0], 0 : similarity.shape[1]] = similarity
            similarity_matrices.append(padded_array)

        similarity_matrix = np.concatenate(similarity_matrices, axis=0)
        similarity_matrix = similarity_matrix[
            0 : embeddings.shape[0], 0 : embeddings.shape[0]
        ]

        similarity_matrix = cosine_similarity(embeddings)
        similarity_matrix -= np.identity(len(similarity_matrix))

        dataset.match(F("max_similarity") > similarity_threshold)
        dataset.tags = ["delete", "has_duplicates"]

        id_map = [s.id for s in dataset.select_fields(["id"])]
        samples_to_remove = set()
        samples_to_keep = set()

        for idx, sample in enumerate(dataset):
            if sample.id not in samples_to_remove:
                # Keep the first instance of two duplicates
                samples_to_keep.add(sample.id)

            dup_idxs = np.where(similarity_matrix[idx] > similarity_threshold)[0]
            for dup in dup_idxs:
                # We kept the first instance so remove all other duplicates
                samples_to_remove.add(id_map[dup])

            if len(dup_idxs) > 0:
                sample.tags.append("has_duplicates")
                sample.save()
            else:
                sample.tags.append("delete")
                sample.save()

        clear_output()

        sidebar_groups = fo.DatasetAppConfig.default_sidebar_groups(dataset)
        for group in sidebar_groups[1:]:
            group.expanded = False
            dataset.app_config.sidebar_groups = sidebar_groups
            dataset.save()
            session = fo.launch_app(dataset)

        print("❗ Wait a minute for the session to load. If it doesn't, read above.")
        print("❗ When it's ready, you'll see a grid of your images.")
        print(
            '❗ On the left side enable "sample tags" to visualize the images marked for deletion.'
        )
        print(
            '❗ You can mark your own images with the "delete" label by selecting them and pressing the tag icon at the top.'
        )
        input("⭕ When you're done, enter something here to save your changes: ")

        print("💾 Saving...")

        marked = [s for s in dataset if "delete" in s.tags]
        dataset.remove_samples(marked)
        previous_folder = images_folder[: images_folder.rfind("/")]
        dataset.export(
            export_dir=os.path.join(images_folder, project_subfolder),
            dataset_type=fo.types.ImageDirectory,
        )

        temp_suffix = "_temp"
        !mv {images_folder} {images_folder}{temp_suffix}
        !mv {images_folder}{temp_suffix}/{project_subfolder} {images_folder}
        !rm -r {images_folder}{temp_suffix}

        session.refresh()
        fo.close_app()
        clear_output()

        print(
            f"\n✅ Removed {len(marked)} images from dataset. You now have {len(os.listdir(images_folder))} images."
        )


proceed3_btn = widgets.Button(
    description="Proceed",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Proceed",
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)
proceed3_btn.on_click(proceed_step_3)

display(
    widgets.HTML("<h2>3️⃣ Curate your images</h2>"),
    widgets.Label(
        "We will find duplicate images with the FiftyOne AI, and mark them with `delete`."
    ),
    widgets.Label(
        "Then, an interactive area will appear below this cell that lets you visualize all your images and manually mark with `delete` to the ones you don't like."
    ),
    widgets.Label(
        "If the interactive area appears blank for over a minute, try enabling cookies and removing tracking protection for the Google Colab website, as they may break it."
    ),
    widgets.Label(
        "Regardless, you can save your changes by sending Enter in the input box above the interactive area."
    ),
    widgets.Label(
        "This is how similar 2 images must be to be marked for deletion. I recommend 0.97 to 0.99:"
    ),
    similarity_threshold_widget,
    proceed3_btn,
)

HTML(value='<h2>3️⃣ Curate your images</h2>')

Label(value='We will find duplicate images with the FiftyOne AI, and mark them with `delete`.')

Label(value="Then, an interactive area will appear below this cell that lets you visualize all your images and…

Label(value='If the interactive area appears blank for over a minute, try enabling cookies and removing tracki…

Label(value='Regardless, you can save your changes by sending Enter in the input box above the interactive are…

Label(value='This is how similar 2 images must be to be marked for deletion. I recommend 0.97 to 0.99:')

FloatText(value=0.0003, description='Similarity threshold:')

Button(description='Proceed', icon='check', style=ButtonStyle(), tooltip='Proceed')

In [22]:
%pip install spandrel tqdm

import shutil
from pathlib import Path

import cv2
import numpy as np
import torch
from spandrel import ImageModelDescriptor, ModelLoader
from tqdm import tqdm

upscale_model_path = Path(main_dir) / "upscale_models"
upscale_models = [p.name for p in upscale_model_path.iterdir()]
upscale_model_widget = widgets.Dropdown(
    options=upscale_models,
    value=upscale_models[0],
    description="Upscale model:",
    disabled=False,
)


def image_to_tensor(img: np.ndarray) -> torch.Tensor:
    img = img.astype(np.float32) / 255.0
    if img.ndim == 2:
        img = np.expand_dims(img, axis=2)
    if img.shape[2] == 1:
        pass
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.transpose(img, (2, 0, 1))
    tensor = torch.from_numpy(img)
    return tensor.unsqueeze(0).cuda()


def tensor_to_image(tensor: torch.Tensor) -> np.ndarray:
    image = tensor.cpu().squeeze().numpy()
    image = np.transpose(image, (1, 2, 0))
    image = np.clip((image * 255.0).round(), 0, 255)
    image = image.astype(np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image


def perform_resize_model(b):
    device = torch.device("cuda")
    model = ModelLoader().load_from_file(
        upscale_model_path / upscale_model_widget.value
    )

    if not isinstance(model, ImageModelDescriptor):
        print("invalid model")
        return

    print(
        model.scale,
        model.input_channels,
        model.output_channels,
        model.tags,
        model.tiling,
    )

    model.cuda()

    input = Path(config_folder) / "orig"
    output = Path(config_folder) / "dataset"

    if not input.exists() and output.exists():
        print(
            "'orig' path does not exists, but 'dataset' exists. rename 'dataset' to 'orig'."
        )
        shutil.move(output, input)

    elif not input.exists:
        print("'orig' directory not found. Abort.")
        return

    if not output.exists():
        output.mkdir()

    paths = [p for p in input.iterdir()]
    try:
        for i in tqdm(paths):
            rel = i.relative_to(input)
            o = output / rel

            if i.is_dir():
                o.mkdir()
                continue

            img = cv2.imread(i, cv2.IMREAD_UNCHANGED)

            if img is None:
                continue

            img_tensor = image_to_tensor(img)
            img_tensor.to(device)

            out_tensor = model(img_tensor)
            out = tensor_to_image(out_tensor)

            cv2.imwrite(o, out)

            del img, img_tensor, out_tensor, out
    finally:
        print("image upscale complete.")
        torch.cuda.empty_cache()
        del model

proceed_resize_model = widgets.Button(description="Proceed")
proceed_resize_model.on_click(perform_resize_model)

display(
    widgets.HTML("<h2>4️⃣ Upscale images using upscale model.</h2>"),
    upscale_model_widget,
    proceed_resize_model,
)

DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/setuptools-70.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


HTML(value='<h2>4️⃣ Upscale images using upscale model.</h2>')

Dropdown(description='Upscale model:', options=('2x-sudo-RealESRGAN.pth', '4x-WTP-UDS-Esrgan.pth', '4x_escale_…

Button(description='Proceed', style=ButtonStyle())

In [16]:
%pip install tqdm

import math
from pathlib import Path

import cv2
from tqdm import tqdm

scale_widget = widgets.FloatSlider(
    value=2.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description="Scale:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".1f",
)

method_widget = widgets.Dropdown(
    options=[
        ("nearest neighbor interpolation", cv2.INTER_NEAREST),
        ("bilinear interpolation", cv2.INTER_LINEAR),
        ("bicubic interpolation", cv2.INTER_CUBIC),
        ("resampling using pixel area relation", cv2.INTER_AREA),
        ("Lanczos interpolation over 8x8 neighborhood", cv2.INTER_LANCZOS4),
        ("Bit exact bilinear interpolation", cv2.INTER_LINEAR_EXACT),
        ("Bit exact nearest neighbor interpolation", cv2.INTER_NEAREST_EXACT),
    ],
    value=cv2.INTER_NEAREST,
    description="Interpolation method:",
    disabled=False,
)


def perform_resize(b):
    input = Path(config_folder) / "orig"
    output = Path(config_folder) / "dataset"

    if not input.exists() and output.exists():
        print(
            "'orig' path does not exists, but 'dataset' exists. rename 'dataset' to 'orig'."
        )
        shutil.move(output, input)

    elif not input.exists:
        print("'orig' directory not found. Abort.")
        return

    if not output.exists():
        output.mkdir()

    paths = [p for p in input.iterdir()]

    for i in tqdm(paths):
        rel = i.relative_to(input)
        o = output / rel

        if i.is_dir():
            o.mkdir()

            continue

        img = cv2.imread(i, cv2.IMREAD_UNCHANGED)
        if img is None:
            continue

        h, w = img.shape[:2]
        h = math.floor(h * scale_widget.value)
        w = math.floor(w * scale_widget.value)

        out = cv2.resize(img, (w, h), interpolation=method_widget.value)
        cv2.imwrite(o, out)

    print("image upscale complete.")


proceed_resize = widgets.Button(description="Proceed")

proceed_resize.on_click(perform_resize)

display(
    widgets.HTML("<h2>4️⃣ Upscale images.</h2>"),
    scale_widget,
    method_widget,
    proceed_resize,
)

DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/setuptools-70.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


HTML(value='<h2>4️⃣ Upscale images.</h2>')

FloatSlider(value=2.0, continuous_update=False, description='Scale:', max=10.0, min=0.1, readout_format='.1f')

Dropdown(description='Interpolation method:', options=(('nearest neighbor interpolation', 0), ('bilinear inter…

Button(description='Proceed', style=ButtonStyle())

In [17]:
method_widget = widgets.Dropdown(
    value="Anime tags", options=["Anime tags", "Photo captions"], description="Method"
)

tag_threshold_widget = widgets.BoundedFloatText(
    value=0.35, min=0.0, max=1.0, step=0.01, description="Tag threshold:"
)

blacklist_tags_widget = widgets.Textarea(
    value="bangs, breasts, multicolored hair, two-tone hair, gradient hair, virtual youtuber, parody, style parody, official alternate costume, official alternate hairstyle, official alternate hair length, alternate costume, alternate hairstyle, alternate hair length, alternate hair color",
    description="Blacklist tags:",
    disabled=False,
)

caption_min_widget = widgets.IntText(
    value=10, description="Caption min:", disabled=False
)

caption_max_widget = widgets.IntText(
    value=75, description="Caption min:", disabled=False
)

proceed4_btn = widgets.Button(
    description="Proceed",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Proceed",
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)

display(
    widgets.HTML("<h2>4️⃣ Tag your images</h2>"),
    widgets.HTML(
        """<p>We will be using AI to automatically tag your images, specifically 
        <a href='https://huggingface.co/SmilingWolf/wd-v1-4-swinv2-tagger-v2'>Waifu Diffusion</a> 
        in the case of anime and <a href='https://huggingface.co/spaces/Salesforce/BLIP'>BLIP</a> 
        in the case of photos.</p>
    """
    ),
    widgets.Label(
        """Giving tags/captions to your images allows for much better training. 
        This process should take a couple minutes."""
    ),
    method_widget,
    widgets.HTML(
        """<b>Anime:</b> The threshold is the minimum level of confidence the tagger must have 
      in order to include a tag. Lower threshold = More tags. Recommended 0.35 to 0.5
    """
    ),
    tag_threshold_widget,
    blacklist_tags_widget,
    widgets.HTML(
        """<b>Photos:</b> The minimum and maximum length of tokens/words in each caption."""
    ),
    caption_min_widget,
    caption_max_widget,
    proceed4_btn,
)

def proceed_step_4(b):
    if "step1_installed_flag" not in globals():
        raise Exception("Please run step 1 first!")

    method = method_widget.value
    tag_threshold = tag_threshold_widget.value
    blacklist_tags = blacklist_tags_widget.value
    caption_min = caption_min_widget.value
    caption_max = caption_max_widget.value

    # %env PYTHONPATH=/env/python
    os.chdir(root_dir)
    kohya = os.path.join(root_dir, "sd-scripts")
    if not os.path.exists(kohya):
        !git clone https://github.com/kohya-ss/sd-scripts {kohya}
    os.chdir(kohya)
    !git checkout tags/v0.8.7
    !git reset --hard
    !pip install -r requirements.txt
    os.chdir(root_dir)

    if "tags" in method:
        if "step4a_installed_flag" not in globals():
            print("\n🏭 Installing dependencies...\n")
            if not get_ipython().__dict__["user_ns"]["_exit_code"]:
                clear_output()
                step4a_installed_flag = True
            else:
                print("❌ Error installing dependencies, trying to continue anyway...")

        print("\n🚶‍♂️ Launching program...\n")

        os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
        # %env PYTHONPATH={kohya}
        !python -i {kohya}/finetune/tag_images_by_wd14_tagger.py \
            --onnx \
            --repo_id=SmilingWolf/wd-swinv2-tagger-v3 \
            --model_dir={root_dir} \
            --thresh={tag_threshold} \
            --batch_size=8 \
            --caption_extension=.txt \
            --force_download \
            {images_folder} \

        if not get_ipython().__dict__["user_ns"]["_exit_code"]:
            print("removing underscores and blacklist...")
            blacklisted_tags = [t.strip() for t in blacklist_tags.split(",")]
            from collections import Counter

            top_tags = Counter()
            for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
                with open(os.path.join(images_folder, txt), "r") as f:
                    tags = [t.strip() for t in f.read().split(",")]
                    tags = [t.replace("_", " ") if len(t) > 3 else t for t in tags]
                    tags = [t for t in tags if t not in blacklisted_tags]
                top_tags.update(tags)
                with open(os.path.join(images_folder, txt), "w") as f:
                    f.write(", ".join(tags))

            # %env PYTHONPATH=/env/python
            # clear_output()
            print(f"📊 Tagging complete. Here are the top 50 tags in your dataset:")
            print("\n".join(f"{k} ({v})" for k, v in top_tags.most_common(50)))


    else:  # Photos
        if "step4b_installed_flag" not in globals():
            print("\n🏭 Installing dependencies...\n")
            
            if not get_ipython().__dict__["user_ns"]["_exit_code"]:
                # clear_output()
                step4b_installed_flag = True
            else:
                print("❌ Error installing dependencies, trying to continue anyway...")

        display("\n🚶‍♂️ Launching program...\n")

        os.chdir(kohya)
        print(f"{images_folder}")
        # %env PYTHONPATH={kohya}
        !python -i {kohya}/finetune/make_captions.py \
            {images_folder} \
            --beam_search \
            --max_data_loader_n_workers=2 \
            --batch_size=8 \
            --min_length={caption_min} \
            --max_length={caption_max} \
            --caption_extension=.txt

        if not get_ipython().__dict__["user_ns"]["_exit_code"]:
            import random

            captions = [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]
            sample = []
            for txt in random.sample(captions, min(10, len(captions))):
                with open(os.path.join(images_folder, txt), "r") as f:
                    sample.append(f.read())

            os.chdir(root_dir)
            # %env PYTHONPATH=/env/python
            # clear_output()
            print(
                f"📊 Captioning complete. Here are {len(sample)} example captions from your dataset:"
            )
            print("".join(sample))

proceed4_btn.on_click(proceed_step_4)

HTML(value='<h2>4️⃣ Tag your images</h2>')

HTML(value="<p>We will be using AI to automatically tag your images, specifically \n        <a href='https://h…

Label(value='Giving tags/captions to your images allows for much better training. \n        This process shoul…

Dropdown(description='Method', options=('Anime tags', 'Photo captions'), value='Anime tags')

HTML(value='<b>Anime:</b> The threshold is the minimum level of confidence the tagger must have \n      in ord…

...

Textarea(value='bangs, breasts, multicolored hair, two-tone hair, gradient hair, virtual youtuber, parody, sty…

HTML(value='<b>Photos:</b> The minimum and maximum length of tokens/words in each caption.')

IntText(value=10, description='Caption min:')

IntText(value=75, description='Caption min:')

Button(description='Proceed', icon='check', style=ButtonStyle(), tooltip='Proceed')

In [18]:
global_activation_tag_widget = widgets.Text(
    placeholder="hatsune miku",
    description="Global activation tag:",
)

remove_tags_widget = widgets.Text(
    description="Remove tags:",
)

search_tags_widget = widgets.Text(description="Search tags:")

replace_with_widget = widgets.Text(description="Replace with:")

search_mode_widget = widgets.Dropdown(
    description="Search Mode", options=["OR", "AND"], value="OR"
)

new_becomes_activation_tag_widget = widgets.Checkbox(
    value=False,
    description="New becomes activation tag:",
)

sort_alphabetically_widget = widgets.Checkbox(
    value=False,
    description="Sort alphabetically:",
)

remove_duplicates_widget = widgets.Checkbox(
    value=False,
    description="Remove duplicates:",
)

proceed5_btn = widgets.Button(
    description="Proceed",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Proceed",
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)

display(
    widgets.HTML("""<h2>5️⃣ Curate your tags</h2>"""),
    widgets.Label(
        "Modify your dataset's tags. You can run this cell multiple times with different parameters."
    ),
    widgets.Label(
        "Put an activation tag at the start of every text file. This is useful to make learning better and activate your Lora easier. Set `keep_tokens` to 1 when training.<p>"
    ),
    widgets.Label(
        'Common tags that are removed such as hair color, etc. will be "absorbed" by your activation tag.'
    ),
    global_activation_tag_widget,
    remove_tags_widget,
    widgets.Label(),
    widgets.Label(
        "In this advanced section, you can search text files containing matching tags, and replace them with less/more/different tags. If you select the checkbox below, any extra tags will be put at the start of the file, letting you assign different activation tags to different parts of your dataset. Still, you may want a more advanced tool for this."
    ),
    search_tags_widget,
    replace_with_widget,
    search_mode_widget,
    new_becomes_activation_tag_widget,
    widgets.Label(
        "These may be useful sometimes. Will remove existing activation tags, be careful."
    ),
    sort_alphabetically_widget,
    remove_duplicates_widget,
    proceed5_btn,
)


def proceed_step_5(b):
    if "step1_installed_flag" not in globals():
        raise Exception("Please run step 1 first!")

    global_activation_tag = global_activation_tag_widget.value
    remove_tags = remove_tags_widget.value

    search_tags = search_tags_widget.value
    replace_with = replace_with_widget.value
    search_mode = search_mode_widget.value
    new_becomes_activation_tag = new_becomes_activation_tag_widget.value

    sort_alphabetically = sort_alphabetically_widget.value
    remove_duplicates = remove_duplicates_widget.value

    def split_tags(tagstr):
        return [s.strip() for s in tagstr.split(",") if s.strip()]

    activation_tag_list = split_tags(global_activation_tag)
    remove_tags_list = split_tags(remove_tags)
    search_tags_list = split_tags(search_tags)
    replace_with_list = split_tags(replace_with)
    replace_new_list = [t for t in replace_with_list if t not in search_tags_list]

    replace_with_list = [t for t in replace_with_list if t not in replace_new_list]
    replace_new_list.reverse()
    activation_tag_list.reverse()

    remove_count = 0
    replace_count = 0

    for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:

        with open(os.path.join(images_folder, txt), "r") as f:
            tags = [s.strip() for s in f.read().split(",")]

        if remove_duplicates:
            tags = list(set(tags))
        if sort_alphabetically:
            tags.sort()

        for rem in remove_tags_list:
            if rem in tags:
                remove_count += 1
                tags.remove(rem)

        if (
            "AND" in search_mode
            and all(r in tags for r in search_tags_list)
            or "OR" in search_mode
            and any(r in tags for r in search_tags_list)
        ):
            replace_count += 1
            for rem in search_tags_list:
                if rem in tags:
                    tags.remove(rem)
            for add in replace_with_list:
                if add not in tags:
                    tags.append(add)
            for new in replace_new_list:
                if new_becomes_activation_tag:
                    if new in tags:
                        tags.remove(new)
                    tags.insert(0, new)
                else:
                    if new not in tags:
                        tags.append(new)

        for act in activation_tag_list:
            if act in tags:
                tags.remove(act)
            tags.insert(0, act)

        with open(os.path.join(images_folder, txt), "w") as f:
            f.write(", ".join(tags))

    if global_activation_tag:
        print(f"\n📎 Applied new activation tag(s): {', '.join(activation_tag_list)}")
    if remove_tags:
        print(f"\n🚮 Removed {remove_count} tags.")
    if search_tags:
        print(f"\n💫 Replaced in {replace_count} files.")
    print("\n✅ Done! Check your updated tags in the Extras below.")


proceed5_btn.on_click(proceed_step_5)

HTML(value='<h2>5️⃣ Curate your tags</h2>')

Label(value="Modify your dataset's tags. You can run this cell multiple times with different parameters.")

Label(value='Put an activation tag at the start of every text file. This is useful to make learning better and…

Label(value='Common tags that are removed such as hair color, etc. will be "absorbed" by your activation tag.'…

Text(value='', description='Global activation tag:', placeholder='hatsune miku')

...

Checkbox(value=False, description='New becomes activation tag:')

Label(value='These may be useful sometimes. Will remove existing activation tags, be careful.')

Checkbox(value=False, description='Sort alphabetically:')

Checkbox(value=False, description='Remove duplicates:')

Button(description='Proceed', icon='check', style=ButtonStyle(), tooltip='Proceed')

In [19]:
# @markdown ### 6️⃣ Ready
# @markdown You should be ready to [train your Lora](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb)!

from IPython.display import Markdown, display

display(
    Markdown(
        f"### 🦀 [Click here to open the Lora trainer](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb)"
    )
)

### 🦀 [Click here to open the Lora trainer](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb)

## *️⃣ Extras

In [20]:
if "step1_installed_flag" not in globals():
    raise Exception("Please run step 1 first!")

# @markdown ### 📈 Analyze Tags
# @markdown Perhaps you need another look at your dataset.
show_top_tags = 50  # @param {type:"number"}

from collections import Counter

top_tags = Counter()

for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
    with open(os.path.join(images_folder, txt), "r") as f:
        top_tags.update([s.strip() for s in f.read().split(",")])

top_tags = Counter(top_tags)
print(f"📊 Top {show_top_tags} tags:")
for k, v in top_tags.most_common(show_top_tags):
    print(f"{k} ({v})")

📊 Top 50 tags:


In [21]:
# @markdown ### 📂 Unzip dataset
# @markdown It's much slower to upload individual files to your Drive, so you may want to upload a zip if you have your dataset in your computer.
zip = "/content/drive/MyDrive/Loras/example.zip"  # @param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/example/dataset"  # @param {type:"string"}

import os
import zipfile

if not os.path.exists("/content/drive"):
    from google.colab import drive

    print("📂 Connecting to Google Drive...")
    drive.mount("/content/drive")

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, "r") as f:
    f.extractall(extract_to)

print("✅ Done")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# @markdown ### 🔢 Count datasets
# @markdown Google Drive makes it impossible to count the files in a folder, so this will show you the file counts in all folders and subfolders.
folder = "/content/drive/MyDrive/Loras"  # @param {type:"string"}

import os

from google.colab import drive

if not os.path.exists("/content/drive"):
    print("📂 Connecting to Google Drive...\n")
    drive.mount("/content/drive")

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
    dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
    images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
    captions = len([f for f in files if f.lower().endswith(".txt")])
    others = len(files) - images - captions
    path = root[folder.rfind("/") + 1 :]
    tree[path] = (
        None if not images else f"{images:>4} images | {captions:>4} captions |"
    )
    if tree[path] and others:
        tree[path] += f" {others:>4} other files"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))

In [ ]:
if "step1_installed_flag" not in globals():
    raise Exception("Please run step 1 first!")

import os

from PIL import Image

Image.MAX_IMAGE_PIXELS = None

# @markdown ### 🖼️ Reduce dataset filesize
# @markdown This will convert all images in the project folder to jpeg, reducing filesize without affecting quality too much. This can also solve some errors.
location = images_folder

for dir in [d[0] for d in os.walk(location)]:
    os.chdir(dir)
    converted = False
    for file_name in list(os.listdir(".")):
        try:
            # Convert png to jpeg
            if file_name.endswith(".png"):
                if not converted:
                    print(f"Converting {dir}")
                    converted = True
                im = Image.open(file_name)
                im = im.convert("RGB")
                new_file_name = os.path.splitext(file_name)[0] + ".jpeg"
                im.save(new_file_name, quality=95)
                os.remove(file_name)
                file_name = new_file_name
            # Resize large jpegs
            if (
                file_name.endswith((".jpeg", ".jpg"))
                and os.path.getsize(file_name) > 2000000
            ):
                if not converted:
                    print(f"Converting {dir}")
                    converted = True
                im = Image.open(file_name)
                im = im.resize((int(im.width / 2), int(im.height / 2)))
                im.save(file_name, quality=95)
            # Rename jpg to jpeg
            if file_name.endswith(".jpg"):
                if not converted:
                    print(f"Converting {dir}")
                new_file_name = os.path.splitext(file_name)[0] + ".jpeg"
                os.rename(file_name, new_file_name)
        except Exception as e:
            print(f"An error occurred while processing {file_name}: {e}")
    if converted:
        print(f"Converted {dir}")

In [ ]:
if "step1_installed_flag" not in globals():
    raise Exception("Please run step 1 first!")

# @markdown ### 🚮 Clean folder
# @markdown Careful! Deletes all non-image files in the project folder.

!find {images_folder} -type f ! \( -name '*.png' -o -name '*.jpg' -o -name '*.jpeg' \) -delete